In [ ]:
# !pip install pyyaml h5py
import os

import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.utils import np_utils
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from numpy import expand_dims
from tensorflow.keras.applications.vgg16 import preprocess_input

In [ ]:
mnist = tf.keras.datasets.mnist

In [ ]:
tf.random.set_seed(7)
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

ndata_train = x_train.shape[0]
ndata_test = x_test.shape[0]


x_train = x_train.reshape((ndata_train,28,28,1))
x_test = x_test.reshape((ndata_test,28,28,1))

xshape = x_train.shape[1:4]

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
model1 = Sequential()

# Convolutional layer 1
model1.add(Conv2D(filters = 16, kernel_size = (2,2), input_shape=xshape, activation='relu'))
model1.add(MaxPooling2D(pool_size=(2,2), strides = 1))

# Convolutional layer 2
model1.add(Conv2D(32,(2,2), activation='relu'))
model1.add(MaxPooling2D(pool_size=(2,2), strides = 1))

# Convolutional layer 3
model1.add(Conv2D(64,(2,2), activation='relu'))
model1.add(MaxPooling2D(pool_size=(2,2), strides = 1))



model1.add(Flatten())

# Feedforward
model1.add(Dense(units= 128, activation='relu'))
model1.add(Dropout(0.1))
model1.add(Dense(units= 128, activation='relu'))
# model1.add(Dropout(0.1))
model1.add(Dense(units=10, activation='softmax'))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005, decay=0.0001, clipvalue=0.5)

model1.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy',
                   metrics= ['accuracy'])

model1_es = EarlyStopping(monitor = 'loss', min_delta = 1e-11, patience = 12, verbose = 1)
model1_rlr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 6, verbose = 1)
model1_mcp = ModelCheckpoint(filepath = r'temp\model1_weights.h5', monitor = 'accuracy',
                             save_best_only = True, verbose = 1)



In [ ]:
history1 = model1.fit(x_train,y_train,epochs=100,validation_split=0.2,batch_size=1000, callbacks=[model1_es, model1_rlr, model1_mcp])

Epoch 1/100
48/48 [==============================] - ETA: 0s - loss: 0.6642 - accuracy: 0.7971
Epoch 00001: accuracy improved from -inf to 0.79715, saving model to temp\model1_weights.h5
48/48 [==============================] - 14s 63ms/step - loss: 0.6642 - accuracy: 0.7971 - val_loss: 0.1997 - val_accuracy: 0.9408 - lr: 5.0000e-04
Epoch 2/100
48/48 [==============================] - ETA: 0s - loss: 0.1633 - accuracy: 0.9507
Epoch 00002: accuracy improved from 0.79715 to 0.95075, saving model to temp\model1_weights.h5
48/48 [==============================] - 3s 69ms/step - loss: 0.1633 - accuracy: 0.9507 - val_loss: 0.0893 - val_accuracy: 0.9743 - lr: 5.0000e-04
Epoch 3/100
48/48 [==============================] - ETA: 0s - loss: 0.0894 - accuracy: 0.9731
Epoch 00003: accuracy improved from 0.95075 to 0.97315, saving model to temp\model1_weights.h5
48/48 [==============================] - 3s 66ms/step - loss: 0.0894 - accuracy: 0.9731 - val_loss: 0.0628 - val_accuracy: 0.9813 - lr: 5.

In [ ]:
print('This model predicts '+str(model1.evaluate(x_test,y_test)[1]*100) +'% of the test data correctly')

313/313 [==============================] - 2s 4ms/step - loss: 0.0310 - accuracy: 0.9918
This model predicts 99.18000102043152% of the test data correctly


In [ ]:
model1.save('my_model.h5') 
model1 = tf.keras.models.load_model('my_model.h5')

In [ ]:
# Retraining with entire dataset using same model
from tensorflow.keras.models import clone_model

model2 = clone_model(model1)


optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005, decay=0.0001, clipvalue=0.5)
model2.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy',
                   metrics= ['accuracy'])

model1_es = EarlyStopping(monitor = 'loss', min_delta = 1e-11, patience = 12, verbose = 1)
model1_rlr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 6, verbose = 1)
model1_mcp = ModelCheckpoint(filepath = r'temp\model1_weights.h5', monitor = 'accuracy',
                             save_best_only = True, verbose = 1)

history2 = model2.fit(x_train,y_train,epochs=100,validation_split=0.0,batch_size=1000, callbacks=[model1_es, model1_rlr, model1_mcp])


Epoch 1/100
60/60 [==============================] - ETA: 0s - loss: 0.5433 - accuracy: 0.8344WARNING:tensorflow:Learning rate reduction is conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy,lr

Epoch 00001: accuracy improved from -inf to 0.83437, saving model to temp\model1_weights.h5
60/60 [==============================] - 4s 58ms/step - loss: 0.5433 - accuracy: 0.8344 - lr: 5.0000e-04
Epoch 2/100
60/60 [==============================] - ETA: 0s - loss: 0.1176 - accuracy: 0.9647WARNING:tensorflow:Learning rate reduction is conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy,lr

Epoch 00002: accuracy improved from 0.83437 to 0.96467, saving model to temp\model1_weights.h5
60/60 [==============================] - 3s 56ms/step - loss: 0.1176 - accuracy: 0.9647 - lr: 5.0000e-04
Epoch 3/100
60/60 [==============================] - ETA: 0s - loss: 0.0724 - accuracy: 0.9780WARNING:tensorflow:Learning rate 

In [ ]:
print('This model predicts '+str(model2.evaluate(x_test,y_test)[1]*100) +'% of the test data correctly')

313/313 [==============================] - 1s 3ms/step - loss: 0.0357 - accuracy: 0.9918
This model predicts 99.18000102043152% of the test data correctly


I am getting upwards of 99% accuracy after expirimenting with multiple model architectures and hyperparameters.  It would be rare to reach 100% accuracy as there is a possibility of numbers being mislabelled, or differences in handwriting making few numbers look very alike which the model is not able to pick.

To increase the accuracy, model can be better tuned or an ensemble of muliple models can be used.

In [ ]:
df  = pd.DataFrame(index=[x for x in range(len(x_test)) ], columns = ['actual','predicted']).fillna(0)
model_prediction = model2.predict(x_test)

for i in range(len(x_test)):
    df.loc[i,'actual'] = y_test[i]
    df.loc[i,'predicted'] = np.argmax(model_prediction[i])

print(df.head())

   actual  predicted
0       7          7
1       2          2
2       1          1
3       0          0
4       4          4


In [ ]:
# MisClassifications 
# Index is the Number and Value is the amount of times it was misclassified
df[df.actual != df.predicted]['actual'].value_counts()

9    23
2    10
6    10
5     8
8     8
4     7
1     4
3     4
7     4
0     4
Name: actual, dtype: int64

In [ ]:
# Change the Misclassified Number below
misclass_num = 9

df[df.actual != df.predicted][df.actual == misclass_num]['predicted'].value_counts() 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


4    9
7    7
5    3
1    2
3    1
8    1
Name: predicted, dtype: int64

What this means is the number 9 which was misclassified 23 times in test data set was read as 4 nine times, 7 seven times etc...

This makes sense as 9 and 4 look kinda alike!!